# TLS

Transport Level Security (TLS) is a cryptographic protocol for establishing secure internet communications. TLS allows us to connect to websites with confidentiality, integrity, and authenticity. When we make a secure connection via TLS, the server and client negotiate a cipher suite. The cipher suite describes the specific cryptographic primitives our secure connection is built on.

The code below establishes a TLS connection to the law school webserver and prints out the cipher suite for the connection.

In [ ]:
import socket
import ssl

hostname = 'law.utexas.edu'
context = ssl.create_default_context()

with socket.create_connection((hostname, 443)) as sock:
    with context.wrap_socket(sock, server_hostname=hostname) as ssock:
        print(ssock.cipher())

This connection uses TLS version 1.2 with cipher suite ECDHE-RSA-AES256-GCM-SHA384. Let's break down this cipher suite and describe each of its components. Our TLS connection uses Elliptic Curve Diffie-Hellman (ECDHE) for key exchange, ensuring forward secrecy. We haven't talked about elliptic curve (EC) cryptography, but this is essentially a more modern variant of the Diffie-Hellman key exchange algorithm. The law school's public key is authenticated using RSA. The messages sent over our connection are all encrypted using AES with a 256-bit key. Remember, block ciphers require a mode of operation. The block cipher mode of operation in this cipher suite is GCM or "Galois Counter Mode". GCM is a more sophisticated version of the counter mode of operation (CTR) we discussed. The integrity of each message is protected with an HMAC based on SHA-384, an SHA-2 variant with a 384-bit hash digest.

We didn't actually do anything with that previous connection. The code above just established a secure connection, then dropped it without sending any data. I'm going to show you what it looks like to request some data. We haven't studied anything about websites or networking or the internet, so I don't want to go too deep into this. This is just to demonstrate that we can "talk" to the web server over this secure connection.

We can ask for a specific resource (e.g. website) with an **HTTP request**. HTTP is a protocol for sending data over the itnernet. I'm going to ask the server to send me the Law School's "About" web page. I "send" my request over the secure TLS connection. We "recv" a response back over the secure TLS connection. I then print that response out.

In [ ]:
import socket
import ssl

hostname = 'law.utexas.edu'
path = 'about/'
context = ssl.create_default_context()

request="GET /%s HTTP/1.1\r\nHost:%s\r\n\r\n" % (path,hostname)

with socket.create_connection((hostname, 443)) as sock:
    with context.wrap_socket(sock, server_hostname=hostname) as ssock:
        ssock.send(request.encode())
        response=ssock.recv(1024)
        print(response.decode())

This is an HTTP response. If you've seen HTML before, you'll recognize the latter part of the response as the start of a HTML document (the contents of a web page.) We requested a web page, and the server is sending its contents to us. The message cuts off before the end, because I only received and printed the first 1024 bytes. I didn't want to print the entire response, because it's very long and quite boring. This is what the raw data looks like that your browser recieves as you browse the web. Your browser makes a series HTTP requests and receives HTTP responses. With TLS, these requests and responses are transmitted over a secure connection. What we're seeing here is the HTTPS protocol in action. **TLS + HTTP = HTTPS**.

Now, hopefully, you have a better idea of how cryptography is used in practice to keep your web browsing secure.

## Exercises

You can complete these exercises using either Python or your web browser to inspect TLS connections.

1) If you make a TLS connection to vgmusic.com, what is the cipher suite? Break down the cipher suite and describe each part.

2) If you make a TLS connection to wikipedia.com, what is the cipher suite? Break down the cipher suite and describe each part. Some of the primitives might be unfamiliar to you. Look up their names.

3) Time for a scavenger hunt. Find some websites that you connect to with strange cipher suites. This might be easier if you use your web browser, since the lawhack server supports fairly standard cipher suites. Your browser probably supports more advanced cipher suites.